
# Example notebook

This example will contain the following examples
- Creating and saving a graph 
- Plotting the graph 
- Executing a node
- Loading a graph from disk

In [1]:
from importlib import reload
%matplotlib
import os
import autodepgraph as adg
from autodepgraph.graph import Graph



Using matplotlib backend: MacOSX


## Creatinga custom graph
A graph can be instantiated

In [2]:
test_graph = Graph('test_graph')
test_graph.add_node('A')
test_graph.add_node('B')
test_graph.add_node('C')

Set the dependencies as a list of nodenames. The nodes which a node `A` depends on are called `A.parents()`. The nodes which depend on a node `B` are called `B.children()`.

In [3]:
test_graph.C.parents(['A', 'B'])
test_graph.B.parents(['A'])

In [4]:
# import time
import matplotlib.pyplot as plt
# Define some custom function to perform a check 
def my_check_function():
    print("Hello")
    plt.pause(2)
    return 'needs calibration'
# Add it to the adg.node_functions module to make it available to nodes
adg.node_functions.check_functions.my_check_function = my_check_function

In [5]:
for node in test_graph.nodes.values():
    node.check_functions(['my_check_function'])
#     node.check_functions([['test_check_needs_calibration'])
    node.calibrate_functions(['test_calibration_True_delayed'])

In [6]:
# Update the plotting monitor (default matplotlib) to show your graph
test_graph.update_monitor()

In [7]:
# Execute your graph 
test_graph.clear_node_state()
test_graph.B.state('needs calibration')
test_graph.A.state('needs calibration')
test_graph.update_monitor()

test_graph.C(verbose=True)

Executing node "C".
Executing node "A".
	Calibrating node A.
	Calibration of node A successful: True
Executing node "B".
	Calibrating node B.
	Calibration of node B successful: True
	Checking node C.
Hello
	Needs C calibration: True
	Node C broken: False
	Calibrating node C.
	Calibration of node C successful: True


'good'

### Rabi model calibrations example

This example shows a bit more realistic graph. 

In [8]:
test_dir = os.path.join(adg.__path__[0], 'tests', 'test_data')
fn = os.path.join(test_dir, 'rabi_sims_example.yaml')


In [9]:
try: 
    Rabi_model_graph = Graph('Rabi_model_graph')
except KeyError: 
    pass
rmg = Rabi_model_graph
rmg.load_graph(fn)

### The layout can be changed by resetting the node pos

In [12]:
rmg._node_pos=None # This resets the positions of nodes 
rmg.update_monitor()

### Reset the state of all nodes

In [13]:
rmg.clear_node_state()
for node in rmg.nodes.values():
    node.state('needs calibration')
rmg.update_monitor()

In [14]:
rmg.High_fidelity_single_qubit_gate(verbose=True)


Executing node "High_fidelity_single_qubit_gate".
Executing node "Amplitude_fine".
Executing node "Motzoi".
Executing node "Amplitude_coarse".
Executing node "frequency_spec".
	Calibrating node frequency_spec.
	Calibration of node frequency_spec successful: True
Executing node "mixer_skewness".
Executing node "mixer_offset".
	Calibrating node mixer_offset.
	Calibration of node mixer_offset successful: True
	Calibrating node mixer_skewness.
	Calibration of node mixer_skewness successful: True
	Calibrating node Amplitude_coarse.
	Calibration of node Amplitude_coarse successful: True
Executing node "frequency_ramsey".
Executing node "SSRO".
	Calibrating node SSRO.
	Calibration of node SSRO successful: True
	Calibrating node frequency_ramsey.
	Calibration of node frequency_ramsey successful: True
	Calibrating node Motzoi.
	Calibration of node Motzoi successful: True
	Calibrating node Amplitude_fine.
	Calibration of node Amplitude_fine successful: True
	Calibrating node High_fidelity_single

'good'

## We can also artificially break one of the nodes and see how the calibration recovers

In [15]:
def check_func_SSRO_dep():
    if rmg.SSRO.state() != 'good':
        return 'bad'
    else:
        return 'good'
adg.node_functions.check_functions.check_func_SSRO_dep = check_func_SSRO_dep

rmg.High_readout_fidelity.check_functions(
    ['check_func_SSRO_dep'])
rmg.Rabi_simulation.check_functions(
    ['check_func_SSRO_dep'])

In [16]:
# rmg.High_readout_fidelity.state('bad')
rmg.SSRO.state('needs calibration')
rmg.update_monitor()

In [17]:
rmg.Rabi_simulation()

'good'

In [18]:
rmg.High_readout_fidelity()

'good'